In [ ]:
import os
os.chdir(os.path.pardir)
from dataset.dataset import Dataset
from evaluation_metrics.diversity_metrics import Topic_diversity
from evaluation_metrics.topic_significance_metrics import KL_uniform
from skopt import gp_minimize, forest_minimize, dummy_minimize
from optimization.optimizer import Optimizer
from skopt.space.space import Real, Integer
import multiprocessing as mp
from models import TorchETM
import torch
import numpy as np

In [ ]:
dataset = Dataset()
dataset.load("preprocessed_datasets/newsgroup/newsgroup_lemmatized_10")

In [ ]:
# Load model
model = TorchETM.ETM_Wrapper()

In [ ]:
model.hyperparameters['num_epochs'] = 6
model.hyperparameters['enc_drop'] = 0.1

In [ ]:
model.partitioning(True)

In [ ]:
model.train_model(dataset, model.hyperparameters, top_words= 10,
                  topic_word_matrix=False,
                    topic_document_matrix=True)

In [ ]:
# Topic diversity
topic_diversity = Topic_diversity()

# KL_Uniform
#kl_uniform = KL_uniform()

In [ ]:
# Define optimization parameters
opt_params = {}
opt_params["n_calls"] = 5
opt_params["minimizer"] = forest_minimize
opt_params["different_iteration"] = 3
opt_params["n_random_starts"] = 5
#opt_params["extra_metrics"] = [kl_uniform] # List of extra metrics
opt_params["n_jobs"] = mp.cpu_count() -1 # Enable multiprocessing
opt_params["verbose"] = True
opt_params["save_path"] = "results" #create folder if it doesn't exist

In [ ]:
# Create search space for optimization
search_space = {
    "num_epochs": Integer(low=1, high=5),
    #"eta": Real(low=0.01, high=5.0)
}

In [ ]:
# Initialize optimizer
optimizer = Optimizer(
    model,
    dataset,
    topic_diversity,
    search_space,
    opt_params)

In [ ]:
# Disable computing of topic document matrix to optimize performance
optimizer.topic_document_matrix = False
optimizer.topic_word_matrix = False

In [ ]:
# Optimize
res = optimizer.optimize()

print(res.hyperparameters) # Best values for the hyperparameters
print(res.function_values) # Score of the optimized metric
print("Optimized metric: "+res.optimized_metric)
